# TD/TME10: SQL2 – Jointures Naturelles, Externes, Sous-requêtes

On utilise  [DuckDB](https://duckdb.org). Voir la [documentation DuckDB SQL](https://duckdb.org/docs/sql/introduction.html)

In [3]:
#installer duckdb
!pip install duckdb

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Démarrer le service DuckDB
import duckdb

db = duckdb.connect(':memory:')


# vérifier que le service fonctionne
r = db.sql("SELECT 'hello' as col")
display(r)

┌─────────┐
│   col   │
│ varchar │
├─────────┤
│ hello   │
└─────────┘

# Créer les tables et charger les données bd_jo_v2_duck.sql

Ce TD/TME utilise les données contenues dans le fichier **bd_jo_v2_duck.sql**

**Attention**: vous pouvez cliquer sur les 3 lignes dans la fenêtre de gauche pour d'afficher les différentes sections du notebook   

In [5]:
with open('bd-jo-v2_duck.sql', 'r',encoding='utf-8') as file:
    data = file.read()
db.execute(data)    

**Consulter la BD**

La commande `SHOW TABLES` retourne le nom des tables.

Note: la fonction *.df()* ajoutée à la fin de l'expression sert à importer le résultat de la requête dans un dataframe pandas afin d'avoir un rendu "ergonomique" dans colab .

In [6]:
db.execute("SHOW TABLES").df()

,name
0,Athlete
1,AthletesEquipe
2,Epreuve
3,Equipe
4,Pays
5,RangEquipe
6,RangIndividuel
7,Sport


La commande `DESCRIBE` retourne une description des attributs d'une table.

Exemple affichant les attributs de la table *Athlete* :

In [7]:
db.execute("DESCRIBE Athlete").df()

,column_name,column_type,null,key,default,extra
0,aid,INTEGER,NO,PRI,None,None
1,nomAth,VARCHAR,YES,None,None,None
2,prenomAth,VARCHAR,YES,None,None,None
3,dateNaissance,DATE,YES,None,None,None
4,codePays,VARCHAR,YES,None,None,None


La clause  `LIMIT N` est ajoutée à la fin d'une requête pour calculer seulement les N premiers tuples du résultat.

Exemple pour afficher 10 tuples de la table *Athlete*:

In [8]:
db.execute("SELECT * FROM Athlete LIMIT 10").df()

,aid,nomAth,prenomAth,dateNaissance,codePays
0,1,BJOERNDALEN,Ole Einar,1974-01-27,NOR
1,2,BJOERGEN,Marit,1980-03-21,NOR
2,3,AN,Victor,1985-11-23,RUS
3,4,PECHSTEIN,Claudia,1972-02-22,GER
4,5,WÜST,Ireen,1986-04-01,NED
5,6,SVENDSEN,Emil Hegle,1985-07-12,NOR
6,7,AMMANN,Simon,1981-06-25,SUI
7,8,KRAMER,Sven,1986-04-23,NED
8,9,SABLIKOVA,Martina,1987-05-27,CZE
9,10,HAMELIN,Charles,1984-04-14,CAN


In [9]:
#Afficher toutels les lignes de la table Athlète
db.execute("select * from athlete").df()

,aid,nomAth,prenomAth,dateNaissance,codePays
0,1,BJOERNDALEN,Ole Einar,1974-01-27,NOR
1,2,BJOERGEN,Marit,1980-03-21,NOR
2,3,AN,Victor,1985-11-23,RUS
3,4,PECHSTEIN,Claudia,1972-02-22,GER
4,5,WÜST,Ireen,1986-04-01,NED
...,...,...,...,...,...
2426,2427,NARUSE,Nobu,NaT,JPN
2427,2428,YOSHIDA,Keishin,NaT,JPN
2428,2429,MIYAZAWA,Hiroyuki,NaT,JPN
2429,2430,LENTING,Akira,NaT,JPN


# TD10: SQL2 – Jointures Naturelles, Externes, Sous-requêtes

On considère le schéma de la base JEUXOLYMPIQUES2014 qui décrit les athlètes et leurs résultats aux
épreuves des sports des Jeux Olympiques d'Hiver Sotchi 2014 :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>


Les attributs qui forment la clé primaire de chaque relation sont soulignés. Les clés étrangères sont
signalées avec une *. Les attributs aid, epid, eqid et sid correspondent aux identifiants des athlètes,
épreuves, équipes et sports et sont utilisés à la fois comme clé primaire ou comme référence (clé
étrangère) vers la relation correspondante.
La relation **PAYS** contient le code et le nom de tous les pays, même si ils n'ont pas participé aux
Jeux Olympiques. Les sports (n-uplets de la relation **SPORT**) sont un ensemble d'épreuves (n-uplets
de la relation **EPREUVE**). Pour chaque épreuve on connaît son nom et les date de début et fin de
l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la
participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table
**RANGINDIVIDUEL** qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les
épreuves par équipe les résultats sont stockés dans la relation **RANGEQUIPE**, alors que l'information
sur le pays de chaque équipe et ses participants et stocké dans les relations **EQUIPE** et
**ATHLETESEQUIPE**. Dans les relations **RANGINDIVIDUEL** et **RANGEQUIPE** l'attribut rang est égal à
null si l'athlète ou l'équipe a été disqualifié.


## Requêtes

### Jointures internes  « INNER JOIN »

#### **Q1**. 
Les noms et les prénoms des athlètes français (nom pays = 'France') (104 lignes).

In [10]:
query="""
select a.nomath, a.prenomath
from athlete a inner join pays p on a.codepays = p.codepays
where p.nomp = 'France'
"""

db.execute(query).df()

,nomAth,prenomAth
0,FOURCADE,Martin
1,LAMY CHAPPUIS,Jason
2,VAULTIER,Pierre
3,CHAPUIS,Jean Frederic
4,BRUNET,Marie Laure
...,...,...
99,Daniel Jean-Paul RICARD,Vincent
100,BOURZAT,Fabian
101,PECHALAT,Nathalie
102,CARRON,Pernelle


#### **Q2** . 
Les épreuves (sport, nom d'épreuve) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (66 lignes).

In [16]:
query="""
select distinct s.nomsp, ep.nomep
from epreuve ep inner join sport s on ep.sid = s.sid
order by s.nomsp, ep.nomep desc
"""

db.execute(query).df()

,nomSp,nomEp
0,Biathlon,"relais 4x7,5km"
1,Biathlon,relais 4x6km
2,Biathlon,Relais mix
3,Biathlon,"7,5km"
4,Biathlon,20km
...,...,...
61,Surf des neiges,Snowboard cross
62,Surf des neiges,Slopestyle
63,Surf des neiges,Slalom parallèle
64,Surf des neiges,Slalom géant parallèle


#### **Q3** . 
Les homonymes (les nom de familles portés par deux athlètes ou plus) (141 lignes)

In [17]:
query="""
select distinct a1.nomath
from athlete a1 inner join athlete a2 on a1.nomath=a2.nomath and a1.prenomath <> a2.prenomath
"""

db.execute(query).df()

,nomAth
0,FOURCADE
1,PARK
2,SEMERENKO
3,MULDER
4,JEAN
...,...
136,HARVEY
137,DVORAK
138,SOBOLEV
139,HERNANDEZ


#### **Q4**. 
Les athlètes ayant participé à (au moins) deux épreuves individuelles (706 lignes).

In [24]:
query="""
select a.nomath, a.prenomath
from athlete a
inner join rangIndividuel r on r.aid = a.aid
group by a.aid, a.nomath, a.prenomath
having count(r.epid) >= 2
"""

db.execute(query).df()

,nomAth,prenomAth
0,SEMERENKO,Valj
1,KUZMINA,Anastazia
2,NOWAKOWSKA-ZIEMNIAK,Weronika
3,SOLEMDAL,Synnoeve
4,PADIAL HERNANDEZ,Victoria
...,...,...
701,KILLICK,Graeme
702,LEGKOV,Alexander
703,NORTHUG,Petter Jr.
704,STAROSTIN,Mark


### Jointures naturelles « NATURAL JOIN » 

#### **Q5**. 
Les noms et les prénoms des athlètes français (nom pays = 'France') (104 lignes).

In [25]:
query="""
select a.nomath, a.prenomath
from athlete a
natural join pays p
where p.nomp = 'France'
"""

db.execute(query).df()

,nomAth,prenomAth
0,FOURCADE,Martin
1,LAMY CHAPPUIS,Jason
2,VAULTIER,Pierre
3,CHAPUIS,Jean Frederic
4,BRUNET,Marie Laure
...,...,...
99,Daniel Jean-Paul RICARD,Vincent
100,BOURZAT,Fabian
101,PECHALAT,Nathalie
102,CARRON,Pernelle


#### **Q6** . 
Les épreuves (sport, nom d'épreuve) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (66 lignes).

In [27]:
query="""
select distinct s.nomsp, ep.nomep
from sport s
natural join epreuve ep
order by s.nomsp, ep.nomep desc
"""

db.execute(query).df()

,nomSp,nomEp
0,Biathlon,"relais 4x7,5km"
1,Biathlon,relais 4x6km
2,Biathlon,Relais mix
3,Biathlon,"7,5km"
4,Biathlon,20km
...,...,...
61,Surf des neiges,Snowboard cross
62,Surf des neiges,Slopestyle
63,Surf des neiges,Slalom parallèle
64,Surf des neiges,Slalom géant parallèle


#### **Q7** . 
Les sports (identifiant et nom) et les épreuves (identifiant et nom) en équipe (25 lignes).

In [29]:
query="""
select distinct s.sid, s.nomsp, ep.epid, ep.nomep
from sport s
natural join epreuve ep
natural join rangEquipe
"""

db.execute(query).df()

,sid,nomSp,epid,nomEp
0,1,Biathlon,10,"relais 4x7,5km"
1,6,Luge,25,Relais par équipes mixte
2,7,Patinage artistique,29,Danse sur glace
3,7,Patinage artistique,30,Par équipes
4,9,Patinage de vitesse sur piste courte,50,Relais 5000m
5,7,Patinage artistique,28,Couples
6,6,Luge,24,Double
7,9,Patinage de vitesse sur piste courte,46,Relais 3000m
8,14,Ski de fond,85,Relais 4x10km
9,5,Hockey sur glace,21,hockey sur glace


#### **Q8** . 
Les noms et les prénoms des athlètes qui ont gagné au moins une médaille en équipe au sport 'Biathlon' (34 lignes).

In [35]:
query="""
select *
from athlete a 
natural join athletesEquipe aEq 
natural join rangEquipe re
natural join Epreuve ep 
natural join sport s
where s.nomsp = 'Biathlon' and re.rang = 1
"""

db.execute(query).df()

,aid,nomAth,prenomAth,dateNaissance,codePays,eqid,epid,rang,sid,nomEp,categorie,dateDebut,dateFin,nomSp
0,1,BJOERNDALEN,Ole Einar,1974-01-27,NOR,36,11,1,1,Relais mix,Mixte,2014-02-19,2014-02-19,Biathlon
1,6,SVENDSEN,Emil Hegle,1985-07-12,NOR,36,11,1,1,Relais mix,Mixte,2014-02-19,2014-02-19,Biathlon
2,30,BERGER,Tora,1981-03-18,NOR,36,11,1,1,Relais mix,Mixte,2014-02-19,2014-02-19,Biathlon
3,37,USTYUGOV,Evgeny,1985-06-04,RUS,17,10,1,1,"relais 4x7,5km",Hommes,2014-02-22,2014-02-22,Biathlon
4,82,ECKHOFF,Tiril,1990-05-21,NOR,36,11,1,1,Relais mix,Mixte,2014-02-19,2014-02-19,Biathlon
5,91,SHIPULIN,Anton,1987-08-21,RUS,17,10,1,1,"relais 4x7,5km",Hommes,2014-02-22,2014-02-22,Biathlon
6,92,SEMERENKO,Vita,1986-01-18,UKR,1,5,1,1,relais 4x6km,Femmes,2014-02-21,2014-02-21,Biathlon
7,107,SEMERENKO,Valj,1986-01-18,UKR,1,5,1,1,relais 4x6km,Femmes,2014-02-21,2014-02-21,Biathlon
8,119,VOLKOV,Alexey,1988-04-05,RUS,17,10,1,1,"relais 4x7,5km",Hommes,2014-02-22,2014-02-22,Biathlon
9,120,DZHYMA,Juliya,1990-09-19,UKR,1,5,1,1,relais 4x6km,Femmes,2014-02-21,2014-02-21,Biathlon


### Jointures externes « LEFT|RIGHT|FULL OUTER JOIN » 

#### **Q9**. 
Pour chaque pays, le nombre de médailles en épreuve individuelle gagnées dans l'ordre décroissant des médailles gagnées (on veut aussi les pays sans médailles) (206 lignes)

In [50]:
query="""
select p.nomp, count(ri.aid) AS nombre_medailles
from pays p
left join athlete a on a.codepays = p.codepays
left join rangIndividuel ri on ri.aid = a.aid AND ri.rang < 4
group by p.nomp
order by nombre_medailles desc
"""

db.execute(query).df()

,nomP,nombre_medailles
0,Norvège,24
1,Pays-Bas,22
2,États-Unis,20
3,Russie,19
4,Canada,17
...,...,...
201,Afghanistan,0
202,Sierra Leone,0
203,Gabon,0
204,Îles Salomon,0


#### **Q10**. 
Tous les numéros d'équipes avec les numéros de leurs membres (NULL si les membres de l'équipe sont inconnus) (1108 lignes)

In [52]:
query="""
select eq.eqid, aEq.aid
from equipe eq left join athletesEquipe aEq on eq.eqid = aEq.eqid
"""

db.execute(query).df()

,eqid,aid
0,1,107.0
1,1,120.0
2,1,92.0
3,1,121.0
4,2,21.0
...,...,...
1103,231,NaN
1104,233,NaN
1105,232,NaN
1106,234,NaN


In [ ]:
query="""

"""

db.execute(query).df()

#### **Q11**. 
Les numéros des équipes dont on ne connaît pas les membres (utilisez la jointure externe) (13 lignes).

In [53]:
query="""
select eq.eqid, aEq.aid
from equipe eq left join athletesEquipe aEq on eq.eqid = aEq.eqid
where aEq.aid is NULL
"""

db.execute(query).df()

,eqid,aid
0,163,NaN
1,229,NaN
2,235,NaN
3,160,NaN
4,162,NaN
5,159,NaN
6,167,NaN
7,230,NaN
8,231,NaN
9,233,NaN


#### **Q12**. 
Tous les noms d'athlètes avec les numéros de leurs équipes (NULL si l'athlète n'appartient à aucune équipe) (2579 lignes).

In [54]:
query="""
select a.nomath, a.prenomath, aEq.eqid
from athlete a left join athletesEquipe aEq on a.aid = aEq.aid
"""

db.execute(query).df()

,nomAth,prenomAth,eqid
0,AN,Victor,244.0
1,WÜST,Ireen,236.0
2,SVENDSEN,Emil Hegle,36.0
3,KRAMER,Sven,238.0
4,MORGENSTERN,Thomas,250.0
...,...,...,...
2574,Aaltonen,Juhamatti,166.0
2575,SICS,Juris,170.0
2576,BENECKEN,Sascha,175.0
2577,PETRULAK,Jozef,179.0


#### **Q13**.
Les epreuves (noms du sport et de l'épreuve) avec un attribut pour chaque catégorie (Hommes, Femmes, Mixte). Par exemple, la requête retourne les nuplets ('Bobsleigh', 'bob à deux', 'Hommes',  'Femmes' , NULL) et ('Luge', 'Double', NULL, NULL, 'Mixte') ? (66 lignes)

In [55]:
query="""
SELECT 
    s.nomsp AS Nom_Sport,
    e.nomep AS Nom_Epreuve,
    MAX(CASE WHEN e.categorie = 'Hommes' THEN e.categorie ELSE NULL END) AS Hommes,
    MAX(CASE WHEN e.categorie = 'Femmes' THEN e.categorie ELSE NULL END) AS Femmes,
    MAX(CASE WHEN e.categorie = 'Mixte' THEN e.categorie ELSE NULL END) AS Mixte
FROM 
    sport s
INNER JOIN 
    epreuve e ON s.sid = e.sid
GROUP BY 
    s.nomsp, e.nomep
"""

db.execute(query).df()

,Nom_Sport,Nom_Epreuve,Hommes,Femmes,Mixte
0,Biathlon,15km,None,Femmes,None
1,Biathlon,relais 4x6km,None,Femmes,None
2,Biathlon,"relais 4x7,5km",Hommes,None,None
3,Patinage artistique,Couples,None,None,Mixte
4,Patinage de vitesse,5000m,Hommes,Femmes,None
...,...,...,...,...,...
61,Ski alpin,Slalom,Hommes,Femmes,None
62,Ski alpin,Super G,Hommes,Femmes,None
63,Ski de fond,10km,None,Femmes,None
64,Ski de fond,30km,None,Femmes,None


In [58]:
query="""
SELECT distinct
    s.nomsp AS Nom_Sport,
    e.nomep AS Nom_Epreuve,
    eh.categorie AS Hommes,
    ef.categorie AS Femmes,
    em.categorie AS Mixte
FROM 
    sport s
left JOIN 
    epreuve e ON s.sid = e.sid
LEFT JOIN 
    epreuve eh ON e.nomep = eh.nomep AND eh.categorie = 'Hommes'
LEFT JOIN 
    epreuve ef ON e.nomep = ef.nomep AND ef.categorie = 'Femmes'
LEFT JOIN 
    epreuve em ON e.nomep = em.nomep AND em.categorie = 'Mixte'

"""

db.execute(query).df()

,Nom_Sport,Nom_Epreuve,Hommes,Femmes,Mixte
0,Biathlon,15km,Hommes,Femmes,None
1,Patinage de vitesse sur piste courte,500m,Hommes,Femmes,None
2,Ski alpin,Super G,Hommes,Femmes,None
3,Ski de fond,"Sprint 1,5km",Hommes,Femmes,None
4,Surf des neiges,Slopestyle,Hommes,Femmes,None
...,...,...,...,...,...
61,Ski alpin,Slalom géant,Hommes,Femmes,None
62,Patinage de vitesse,10000m,Hommes,None,None
63,Ski de fond,30km,None,Femmes,None
64,Ski de fond,Relais 4x5km,None,Femmes,None


### Requêtes dans la clause FROM et top-k

#### **Q14**. 
Le nom et l'age actuel des 10 athlètes les plus jeunes (utiliser *date_sub*). 

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q15**. 
Le nom et l'age actuel des 10 athlètes les plus âgés. 

In [ ]:
query="""

"""
db.execute(query).df()

#### **Q16**. 
Le nombre moyen d'athlètes par pays (avec group by). Aide : compter le nombre
d’athlètes dans chaque pays (ayant au moins un athlète), puis faire la moyenne. Résultat (1 ligne) : 27,625

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q17**. 
L’eqid de la ou des équipes qui sont composées du plus d’athlètes pour ces JO.
 Résultats (3 lignes) : 164 ; 165 ; 166 

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q18**.
Le nombre d'épreuves en individuel où il y a eu au moins 100 participants. Résultat (1 ligne ) : 2

In [ ]:
query="""

"""

db.execute(query).df()

# TME10: SQL2 – Jointures Naturelles, Externes, Sous-requêtes

Ce TME utilise les données contenues dans le fichier **bd_jo_v2_duck.sql**


## Requêtes

### Jointures naturelles « NATURAL JOIN » et Jointures Internes  «INNER JOIN »  

#### **Q1-a)**. 
Les noms des épreuves individuelles (identifiants et noms) et les noms des sports correspondants (identifiants et noms), avec Natural Join (73 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q1-b)**. 
Les noms des épreuves individuelles (identifiants et noms) et les noms des sports correspondants (identifiants et noms), avec Inner Join (73 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q2-a)**.
Les noms des sports auxquels LESSER Erik a participé en individuel avec Natural Join (Résultats 2 lignes: Biathlon, Ski de fond).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q2-b)**.
Les noms des sports auxquels LESSER Erik a participé en individuel avec Inner Join (Résultats 2 lignes: Biathlon, Ski de fond).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q3-a)**. 
Le nom et prénom des athlètes qui ont gagné la médaille d'or dans l'épreuve par équipe 'relais 4x6km' de 'Biathlon' de 'Femmes' avec Natural Join (Résultat : SEMERENKO Vita, SEMERENKO Valj, DZHYMA Juliya, PIDHRUSHNA Olena).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q3-b)**. 
Le nom et prénom des athlètes qui ont gagné la médaille d'or dans l'épreuve par équipe 'relais 4x6km' de 'Biathlon' de 'Femmes' avec Inner Join (Résultat : SEMERENKO Vita, SEMERENKO Valj, DZHYMA Juliya, PIDHRUSHNA Olena).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q4**. 
La liste des pays, avec les épreuves (individuels et en équipe) et les rangs obtenus (293 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q5**. 
Les athlètes qui sont plus jeunes que LESSER Erik, avec Inner join (132 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

### Jointures Externes  « LEFT|RIGHT|FULL OUTER JOIN »   

#### **Q6**. 
Les noms de tous les athlètes avec les numéros des équipes (le numéro est NULL si l'athlète n'appartient à aucune équipe et le nom est NULL si les membres d'une équipe sont inconnus) (2592 lignes).


In [ ]:
query="""

"""

db.execute(query).df()

###  Sous-requêtes dans FROM/SELECT  

#### **Q7**. 
Le tableau complet des résultats (sport, épreuve, catégorie, athlète, rang) trié par sport, épreuve, catégorie et rang (3780 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q8**. 
Pour chaque pays, le nom et le nombre de médailles gagnées (en individuel ou en équipe). Conseil : utilisez la requête 1.4 comme sous-requête dans  FROM (25 lignes)

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q9**. 
Pour chaque pays, le nombre de médailles d'or gagnées en équipe ou en individuel (20 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

#### **Q10**. 
Pour chaque pays, le nombre de médailles d'or, le nombre de médaille d'argent et le nombre de médaille de bronze gagnées. Conseil : utilisez des sous-requêtes dans  SELECT (206 lignes).

In [ ]:
query="""

"""

db.execute(query).df()

Sous-requête dans FROM (on n'obtient que les pays avec au moins une médaille d'or, d'argent et de bronze): 17 lignes 

In [ ]:
query="""

"""

db.execute(query).df()

###  Requêtes top-k  

#### **Q11**. 
Les dix épreuves les plus longues.

In [ ]:
query="""

"""

db.execute(query).df()